# <font color="red">CMPT318 PROJECT</font>

In [1]:
import pandas as pd
import numpy as np
from scipy import misc
from skimage.io import imread_collection
import re
import glob

## <font color="blue"> I. LOAD THE DATA </font>

### i. Load Webcam Images

In [2]:
image_collection = imread_collection('katkam-scaled/*.jpg')
images_np = np.array(image_collection)

In [3]:
# IMAGE IN NUMPY ARRAY.
# images_np
# images_np.shape

In [4]:
images_df = pd.DataFrame(
    images_np.reshape(images_np.shape[0],images_np.shape[1] * images_np.shape[2] * images_np.shape[3]))

In [5]:
# IMAGE IN PANDAS DATAFRAME, AND BEEN RESHAPED.
# images_df
# images_df.shape

## ii. Load Weather Observations Data

In [6]:
#reference: for load multiple file from forder: https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
weather_obv_filenames = glob.glob('yvr-weather/*.csv')
weather_obv_df = []

for filename in weather_obv_filenames:
    weather_obv_df.append(pd.read_csv(filename, skiprows=15)) # first 15 rows are general information, which is not useful data.

weather_obv_df = pd.concat(weather_obv_df).reset_index()
# pd.read_csv('weather-51442-201607.csv', skiprows=15)

In [7]:
# WEATHER OBSERVATIONS DATAFRAME.
# weather_obv_df

## <font color="blue">II. CLEAN&PREPARE DATA</font>

### i. Clean Webcam Images

In [8]:
# GET IMAGE FILE_NAME
image_filenames = np.array(image_collection.files)
# image_filenames[0]

In [9]:
# extract image shoot date form filename
re_image_date = re.compile(r'katkam-\d\d\d\d\d\d\d\d\d\d\d\d\d\d')
def get_image_time(path):
    matches = re_image_date.findall(path)
    if matches:
        # preocess match learned from https://docs.python.org/2/library/re.html
        # take the last match which will be file name
        result = matches[-1]
        return (int(result[-14:-10]), int(result[-10:-8]), int(result[-8:-6]), result[-6:-4] + ':' + result[-4:-2])
    else:
        return "wrong input file name format"
get_image_time = np.vectorize(get_image_time)
images_date = get_image_time(image_filenames)

In [10]:
# EXTRACTED IMAGE DATE
# images_date

In [11]:
# add relevant columns
images_df['Year'] = images_date[0]
images_df['Month'] = images_date[1]
images_df['Day'] = images_date[2]
images_df['Time'] = images_date[3]

In [12]:
# IMAGE DATA WITH DATE TIME
# images_df['Time']

## ii. Clean Weather Observations Data

In [13]:
# DROP UNCESSARY COLUMNS
cleaned_weather_obv = weather_obv_df.drop(['index','Data Quality'], axis=1)
cleaned_weather_obv = cleaned_weather_obv.drop(['Temp Flag', 'Stn Press Flag','Wind Chill Flag', 'Hmdx Flag', 'Visibility Flag', 'Wind Spd Flag', 'Wind Dir Flag', 'Rel Hum Flag', 'Dew Point Temp Flag'], axis=1)

# type(weather_obv_df['Year'][0])

In [14]:
# SPLIT cleaned_weather INTO TWO DATAFRAME, ONE WITH NaN WEATHER COLUMN, ONE WITH NOT NaN COLUMN
data_whoseWeather_IsNaN = cleaned_weather_obv[~ cleaned_weather_obv.Weather.notnull()] # weather with Nan
main_training_data = cleaned_weather_obv[cleaned_weather_obv.Weather.notnull()] # weather without Nan

In [15]:
# TWO SPLIT DATAFRAME
# data_whoseWeather_IsNaN
# main_training_data

In [16]:
main_training_data_withoutHW = main_training_data.drop(['Hmdx', 'Wind Chill'],axis=1)
final_data = main_training_data_withoutHW.dropna()
# main_training_data = cleaned_data.copy()

In [18]:
# final_data

## iii. Join Cleaned Webcam Image and Cleaned Weather obserbations Data Together

In [ ]:
merged_data = final_data.merge(right = images_df, on = ['Year', 'Month', 'Day', 'Time'], how = 'inner')